# LEPL1202 - APP0 - Partie B

Bienvenue sur le Notebook interactif de l'APP0 - Partie B.

## Qu'est qu'un Notebook ?

Les Notebooks (aussi appelés Jupyter Notebooks) sont des documents permettant combiner code et du texte (Markdown), le tout dans des cellules individuelles. Cette cellule est entièrement consituée de texte et la prochaine est une cellule de code (Python). Pour les plus curieux, nous vous invitons à aller sur [leur site](https://jupyter.org/).

Pour modifier une cellule, il suffit de double-cliquer dessus. Pour exécuter une cellule, il existe plusieurs façons de faire:

1. cliquer sur <span class="kb"><kbd>Shift-Enter</kbd></span>;
2. cliquer sur l'icône <i class="fa-step-forward fa"></i> (*"run this cell"*), à gauche d'une cellule;
3. cliquer sur l'icône <i class="fa-play fa"></i><span class="toolbar-btn-label">Run</span>;
4. ou de choisir l'une des options du menu `Cell` (tout en haut).

<b>Attention :</b> même si les cellules de code s'exécutent individuellement, vous pouvez réutiliser des fonctions ou variables définies dans une cellule partout après. Notez que l'ordre d'exécution cellules est pris en compte. Ceci est indiqué par le numéro entre crochet (example : <bdi>In</bdi>&nbsp;[1]). Si vous avez un doute sur la bonne exécution de vos cellules, et que quelque chose ne fonctionne pas comme attendu, cliquez sur le bouton <i class="fa-forward fa"></i> and *"Restart and Run All Cells"*.

## Que dois-je faire ?

Ce notebook doit être lu et complété en parallèle de l'énoncé PDF.

La plupart de temps, il vous faudra juste **lire** et **intéragir** avec les quelques boutons / glissières qui rendent les graphiques interactifs.

Quand du code de votre part est attendu, il sera indiqué avec des commentaires.

<b>Note :</b> certaines cellules de code ont été volontairement cachées, pour gagner en visibilité. Il faut cependant bien les exécuter (<i class="fa-step-forward fa"></i>). Pour afficher leur contenu, dans le menu, `View -> Cell Toolbar -> Hide code`.

# 1. Import des modules nécessaires

Comme dans beaucoup de projets Python, nous allons utiliser des modules externes.

Parmi ceux-ci, celui avec lequel vous allez intéragir ce nomme [**NumPy**](https://numpy.org/). Par convention, on l'importe sous l'alias `np`:

```python
import numpy as np
```

**NumPy** est un module permettant d'effectuer des opérations mathématiques classiques sur des vecteurs / matrices multi-dimensionnelles, de manière très performante.

Dès lors, la plupart des fonctions habituelles du modules se retrouve dans **NumPy**:

* `math.cos(x)` devient `np.cos(x)`;
* le produit classique (1 à 1) de `x` avec `y` se fait via `x * y`;
* le produit scalaire (*"dot-product"*) de `x` et `y` se fait via `np.dot(x, y)`;
* et bien plus, voir [leur guide](https://numpy.org/devdocs/user/quickstart.html).

In [ ]:
import uuid

import numpy as np
import plotly.graph_objects as go

from ipywidgets import interact
from typing import Tuple, Union

# 2. Définition des courbes Rouge et Bleue

In [ ]:
def R(
    x: np.ndarray,
    z_a: float = 1,
    z_b: float = 0,
    L: float = 1,
    return_dydx: bool = False,
) -> Union[Tuple[np.ndarray, np.ndarray]]:
    """
    Courbe R(ouge), évaluée aux points x.

    x: vecteur des abscisses, de 0 à L, de longueur n
    z_a, z_b, L: paramètres de la courbe
    return_dydx: si True, retourne aussi la dérivée première de la courbe
    """
    y = z_b + (z_a - z_b) * ((x - L) / L) ** 2

    if return_dydx:
        dydx = (2 / L) * (z_a - z_b) * ((x - L) / L)
        return y, dydx
    else:
        return y


def B(
    x: np.ndarray,
    z_a: float = 1,
    z_b: float = 0,
    L: float = 1,
    return_dydx: bool = False,
) -> Union[Tuple[np.ndarray, np.ndarray]]:
    """
    Courbe B(leue), évaluée aux points x.

    x: vecteur des abscisses, de 0 à L, de longueur n
    z_a, z_b, L: paramètres de la courbe
    return_dydx: si True, retourne aussi la dérivée première de la courbe
    """
    y = z_a - (z_a - z_b) * (x / L)

    if return_dydx:
        dydx = (z_b - z_a) * np.ones_like(x) / L
        return y, dydx
    else:
        return y

In [ ]:
trace_R = go.Scatter(
    x=[],
    y=[],
    fill="tozeroy",
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=[],
    y=[],
    fill="tonexty",
    name="B",
    mode="lines",
    line_color="blue",
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_1 = go.FigureWidget(data=[trace_R, trace_B, trace_Z])


@interact(L=(1, 10, 1), z_a=(0, 1, 0.1), z_b=(0, 1, 0.1))
def update(L=1, z_a=1, z_b=0):
    with fig_1.batch_update():
        x = np.linspace(0, L)
        fig_1.data[0].x = x
        fig_1.data[0].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_1.data[1].x = x
        fig_1.data[1].y = B(x, L=L, z_a=z_a, z_b=z_b)
        fig_1.data[2].x = [0, L]
        fig_1.data[2].y = [z_a, z_b]
        fig_1.data[2].uid = str(uuid.uuid4())  # This forces the text to be updated


fig_1

In [ ]:
def aire(x: np.ndarray, y: np.ndarray) -> float:
    """
    Calculer l'aire sous la courbe, en utilisant l'intégrale
    par somme des rectangles (ou [BONUS] trapèzes).

    x: vecteur des abscisses, de 0 à L, de longueur n
    y: vecteur de ordonnées, de longeur n
    """
    dx = x[1] - x[0]  # Base d'un rectangle

    # return dx * np.sum(y[:-1])
    return np.trapz(y, dx=dx)
    return 0  # À changer !

In [ ]:
g = np.array([0, -9.81])  # m / s^2
m = 1  # kg
F = m * g  # N

In [ ]:
trace_R = go.Scatter(
    x=[],
    y=[],
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=[],
    y=[],
    name="B",
    mode="lines",
    line_color="blue",
)

trace_R_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="red",
    name="Aire approx. de R",
    opacity=0.5,
)

trace_B_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="blue",
    name="Aire approx. de B",
    opacity=0.5,
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_2 = go.FigureWidget(
    data=[trace_B_approx, trace_R_approx, trace_R, trace_B, trace_Z]
)


@interact(L=(1, 10, 1), z_a=(0, 1, 0.1), z_b=(0, 1, 0.1), n=(4, 100, 4))
def update(L=1, z_a=1, z_b=0, n=4):
    with fig_2.batch_update():
        x = np.linspace(0, L)
        fig_2.data[2].x = x
        fig_2.data[2].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_2.data[3].x = x
        fig_2.data[3].y = B(x, L=L, z_a=z_a, z_b=z_b)
        x, dx = np.linspace(0, L, n, retstep=True)
        y_R, dydx_R = R(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        y_B, dydx_B = B(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        fig_2.data[1].x = x[:-1]
        fig_2.data[1].y = y_R[:-1]
        fig_2.data[1].width = dx
        fig_2.data[0].x = x[:-1]
        fig_2.data[0].y = y_B[:-1]
        fig_2.data[0].width = dx
        fig_2.data[-1].x = [0, L]
        fig_2.data[-1].y = [z_a, z_b]
        fig_2.data[-1].uid = str(uuid.uuid4())  # This forces the text to be updated
        aire_R = aire(x, y_R)
        aire_B = aire(x, y_B)
        fig_2.layout.title.text = f"Aire R :{aire_R:.4} / Aire B : {aire_B:.4}"


fig_2

In [ ]:
def travail(x: np.ndarray, y: np.ndarray, dydx: np.ndarray, F: np.ndarray) -> float:
    """
    Calcul le travail effectué par F le long d'une courbe.

    x: vecteur des abscisses, de 0 à L, de longueur n
    y: vecteur de ordonnées, de longeur n
    dydx: vecteur des dérivées % à x, de longueur n
    F: vecteur force, de longeur 2, i.e., F = [F_x, F_y]
    """
    dx = x[1] - x[0]  # Base d'un rectangle
    x = x[:-1]
    n = len(x)
    df = np.zeros((2, n))
    df[0, :] = np.ones(n)
    df[1, :] = dydx[:-1]

    F_proj = np.dot(F, df)

    # return np.sum(F_proj[:-1] * dx)
    return np.trapz(F_proj, dx=dx)

In [ ]:
trace_R = go.Scatter(
    x=[],
    y=[],
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=[],
    y=[],
    name="B",
    mode="lines",
    line_color="blue",
)

trace_R_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="red",
    name="Aire approx. de R",
    opacity=0.5,
)

trace_B_approx = go.Bar(
    x=[],
    y=[],
    offset=0.0,
    width=1,
    marker_color="blue",
    name="Aire approx. de B",
    opacity=0.5,
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_3 = go.FigureWidget(
    data=[trace_B_approx, trace_R_approx, trace_R, trace_B, trace_Z]
)


@interact(L=(1, 10, 1), z_a=(0, 1, 0.1), z_b=(0, 1, 0.1), n=(4, 100, 4))
def update(L=1, z_a=1, z_b=0, n=4):
    with fig_3.batch_update():
        x = np.linspace(0, L)
        fig_3.data[2].x = x
        fig_3.data[2].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_3.data[3].x = x
        fig_3.data[3].y = B(x, L=L, z_a=z_a, z_b=z_b)
        x, dx = np.linspace(0, L, n, retstep=True)
        y_R, dydx_R = R(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        y_B, dydx_B = B(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        fig_3.data[1].x = x[:-1]
        fig_3.data[1].y = y_R[:-1]
        fig_3.data[1].width = dx
        fig_3.data[0].x = x[:-1]
        fig_3.data[0].y = y_B[:-1]
        fig_3.data[0].width = dx
        fig_3.data[-1].x = [0, L]
        fig_3.data[-1].y = [z_a, z_b]
        fig_3.data[-1].uid = str(uuid.uuid4())  # This forces the text to be updated
        travail_R = travail(x, y_R, dydx_R, F)
        travail_B = travail(x, y_B, dydx_B, F)
        fig_3.layout.title.text = f"Travail R : {travail_R:.4} / Travail B : {travail_B:.4}"


fig_3